In [1]:
import os 

import matplotlib.pyplot as plt
import networkx as nx

data_dir = '../data'

In [2]:
dataset = 'node2vec_PPI'
node_file = 'node2vec_PPI_labels.txt'
edge_file = 'node2vec_PPI.edgelist'

In [3]:
def read_node_labels(filename):
    fin = open(filename, 'r')
    node_list = []
    labels = []
    while 1:
        l = fin.readline()
        if l == '':
            break
        vec = l.strip().split()
        node_list.append(vec[0])
        labels.append(vec[1:])
    fin.close()
    print('Nodes with labels: %s'%len(node_list))
    return node_list, labels


def read_for_SVD(filename, weighted=False):
    if weighted:
        G = nx.read_weighted_edgelist(filename)
    else:
        G = nx.read_edgelist(filename)
    return G

In [4]:
node_list, labels = read_node_labels(os.path.join(data_dir, dataset, node_file))
node_dict = [(int(node), {'label': int(node), 'value': label}) for node, label in zip(node_list, labels)]  
print(node_dict) 

G = read_for_SVD(os.path.join(data_dir, dataset, edge_file), weighted=True)
G.add_nodes_from(node_dict)

Nodes with labels: 3890
[(1, {'label': 1, 'value': ['6', '16', '37', '39', '46']}), (2, {'label': 2, 'value': ['30']}), (3, {'label': 3, 'value': ['5']}), (4, {'label': 4, 'value': ['30']}), (5, {'label': 5, 'value': ['2', '9']}), (6, {'label': 6, 'value': ['3', '22', '25']}), (7, {'label': 7, 'value': ['42']}), (8, {'label': 8, 'value': ['3']}), (9, {'label': 9, 'value': ['28', '49']}), (10, {'label': 10, 'value': ['27']}), (11, {'label': 11, 'value': ['10']}), (12, {'label': 12, 'value': ['46']}), (13, {'label': 13, 'value': ['27', '34']}), (14, {'label': 14, 'value': ['27', '34']}), (15, {'label': 15, 'value': ['28']}), (16, {'label': 16, 'value': ['6', '34']}), (17, {'label': 17, 'value': ['21']}), (18, {'label': 18, 'value': ['32', '40']}), (19, {'label': 19, 'value': ['24', '26', '27', '30']}), (20, {'label': 20, 'value': ['27']}), (21, {'label': 21, 'value': ['8']}), (22, {'label': 22, 'value': ['49']}), (23, {'label': 23, 'value': ['26', '34', '44']}), (24, {'label': 24, 'value

In [5]:
G.nodes
G.nodes[1]

{'label': 1, 'value': ['6', '16', '37', '39', '46']}

# Graph Augmentation

In [10]:
import GCL.augmentors as A

In [11]:
aug = A.Compose([
    A.EdgeRemoving(pe=0.3), 
    A.FeatureMasking(pf=0.3)
])

In [12]:
aug = A.RandomChoice(
    [
        A.RWSampling(num_seeds=1000, walk_length=10),
        A.NodeDropping(pn=0.1),
        A.FeatureMasking(pf=0.1),
        A.EdgeRemoving(pe=0.1)
    ],
    num_choices=1
)

In [2]:
import torch
import os.path as osp
import GCL.losses as L

from torch import nn
from tqdm import tqdm
from torch.optim import Adam
from GCL.eval import get_split, LREvaluator
from GCL.models import SingleBranchContrast
from torch_geometric.nn import SAGEConv
from torch_geometric.nn.inits import uniform
from torch_geometric.loader import NeighborSampler
from torch_geometric.datasets import Reddit
"""
From the "Inductive Representation Learning on Large Graphs" paper
<https://arxiv.org/abs/1706.02216>

We constructed a graph dataset from Reddit posts made in the month of September, 2014. 
The node label in this case is the community, or “subreddit”, that a post belongs to. 
We sampled 50 large communities and built a post-to-post graph, connecting posts if the 
same user comments on both. 
In total this dataset contains 232,965 posts with an average degree of 492. We use the 
first 20 days for training and the remaining days for testing (with 30% used for validation). 
For features, we use off-the-shelf 300-dimensional GloVe CommonCrawl word vectors [27]; for 
each post, we concatenated 
(i) the average embedding of the post title, 
(ii) the average embedding of all the post’s comments 
(iii) the post’s score, and 
(iv) the number of comments made on the post.
"""


class GConv(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super(GConv, self).__init__()
        self.layers = torch.nn.ModuleList()
        self.activations = torch.nn.ModuleList()
        for i in range(num_layers):
            if i == 0:
                self.layers.append(SAGEConv(input_dim, hidden_dim))
            else:
                self.layers.append(SAGEConv(hidden_dim, hidden_dim))
            self.activations.append(nn.PReLU(hidden_dim))

    def forward(self, x, adjs):
        for i, (edge_index, _, size) in enumerate(adjs):
            x_target = x[:size[1]]
            x = self.layers[i]((x, x_target), edge_index)
            x = self.activations[i](x)
        return x


class Encoder(torch.nn.Module):
    def __init__(self, encoder, hidden_dim):
        super(Encoder, self).__init__()
        self.encoder = encoder
        self.project = torch.nn.Linear(hidden_dim, hidden_dim)
        uniform(hidden_dim, self.project.weight)

    @staticmethod
    def corruption(x, edge_index):
        return x[torch.randperm(x.size(0))], edge_index

    def forward(self, x, edge_index):
        z = self.encoder(x, edge_index)
        g = self.project(torch.sigmoid(z.mean(dim=0, keepdim=True)))
        zn = self.encoder(*self.corruption(x, edge_index))
        return z, g, zn


def train(encoder_model, contrast_model, data, dataloader, optimizer):
    encoder_model.train()
    total_loss = total_examples = 0
    for batch_size, node_id, adjs in dataloader:
        adjs = [adj.to('cuda') for adj in adjs]
        optimizer.zero_grad()
        z, g, zn = encoder_model(data.x[node_id], adjs)
        loss = contrast_model(h=z, g=g, hn=zn)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * z.shape[0]
        total_examples += z.shape[0]
    return total_loss / total_examples


def test(encoder_model, data, dataloader):
    encoder_model.eval()
    zs = []
    for i, (batch_size, node_id, adjs) in enumerate(dataloader):
        adjs = [adj.to('cuda') for adj in adjs]
        z, _, _ = encoder_model(data.x[node_id], adjs)
        zs.append(z)
    x = torch.cat(zs, dim=0)

    split = get_split(num_samples=x.size()[0], train_ratio=0.1, test_ratio=0.8)
    result = LREvaluator()(x, data.y, split)
    return result


In [3]:
import torch.multiprocessing
torch.multiprocessing.set_sharing_strategy('file_system')

device = torch.device('cuda')
path = osp.join(osp.expanduser('~'), 'datasets', 'Reddit')
dataset = Reddit(path)
data = dataset[0].to(device)

data # Data(x=[232965, 602], edge_index=[2, 114615892], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965])

Data(x=[232965, 602], edge_index=[2, 114615892], y=[232965], train_mask=[232965], val_mask=[232965], test_mask=[232965])

In [26]:
data.x
# # tensor([[ 1.2334,  9.0430, -0.9233,  ..., -0.2579,  0.3112, -0.3772],
# #         [-0.1386, -0.2022,  0.1277,  ...,  0.1563,  0.1048, -0.6534],
# #         [-0.1330, -0.1962, -0.0296,  ...,  0.0358,  0.2864,  0.2744],
# #         ...,
# #         [-0.0614, -0.2022,  0.9698,  ...,  1.1064, -1.4323, -0.2398],
# #         [-0.1606, -0.2022, -0.0892,  ...,  0.7440, -0.5046, -2.2288],
# #         [ 0.0929,  0.2822,  0.1768,  ...,  0.2196,  0.5967,  0.5588]],
# #        device='cuda:0')
data.x.shape 
# torch.Size([232965, 602])
data.x[0]
# tensor([ 1.2334e+00,  9.0430e+00, -9.2328e-01,  1.0542e+00, -1.1125e+00,
# ...
#         -1.7817e-01,  9.5668e-02, -4.9747e-01, -4.4391e-01, -2.5790e-01,
#          3.1119e-01, -3.7721e-01], device='cuda:0')
data.x[0].shape
# torch.Size([602])

data.edge_index 
# tensor([[     0,      0,      0,  ..., 232964, 232964, 232964],
#         [   242,    249,    524,  ..., 231806, 232594, 232634]],
#        device='cuda:0')
data.edge_index.shape 
# torch.Size([2, 114615892])
data.edge_index[0] 
# tensor([     0,      0,      0,  ..., 232964, 232964, 232964], device='cuda:0')
data.edge_index[0].shape 
# torch.Size([114615892])

data.y
# tensor([30, 17, 18,  ...,  3, 13, 13], device='cuda:0')
data.y.shape
# torch.Size([232965])
torch.unique(data.y)
# tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
#         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
#         36, 37, 38, 39, 40], device='cuda:0')

data.train_mask
# tensor([False,  True, False,  ...,  True,  True, False], device='cuda:0')

data.val_mask
# tensor([False, False, False,  ..., False, False, False], device='cuda:0')

data.test_mask
# tensor([ True, False,  True,  ..., False, False,  True], device='cuda:0')

data.edge_index.shape

torch.Size([2, 114615892])

In [25]:
train_loader = NeighborSampler(data.edge_index, node_idx=None,
                                sizes=[10, 10, 25], batch_size=128,
                                shuffle=True, num_workers=32)
test_loader = NeighborSampler(data.edge_index, node_idx=None,
                                sizes=[10, 10, 25], batch_size=128,
                                shuffle=False, num_workers=32)

gconv = GConv(input_dim=dataset.num_features, hidden_dim=512, num_layers=3).to(device)
encoder_model = Encoder(encoder=gconv, hidden_dim=512).to(device)
contrast_model = SingleBranchContrast(loss=L.JSD(), mode='G2L').to(device)

optimizer = Adam(encoder_model.parameters(), lr=0.0001)

c:\Users\alexa\anaconda3\envs\gcl-py311-cu121\Lib\site-packages\torch\utils\data\dataloader.py:558: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [26]:
with tqdm(total=30, desc='(T)') as pbar:
    for epoch in range(1, 31):
        loss = train(encoder_model, contrast_model, data, train_loader, optimizer)
        pbar.set_postfix({'loss': loss})
        pbar.update()

test_result = test(encoder_model, data, test_loader)
print(f'(E): Best test F1Mi={test_result["micro_f1"]:.4f}, F1Ma={test_result["macro_f1"]:.4f}')

(T):   0%|          | 0/30 [11:17<?, ?it/s]


KeyboardInterrupt: 

In [28]:
pip install numba

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
    --------------------------------------- 0.0/2.6 MB 991.0 kB/s eta 0:00:03
   ------ --------------------------------- 0.4/2.6 MB 5.3 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.6 MB 8.2 MB/s eta 0:00:01
   --------------------------- ------------ 1.8/2.6 MB 10.7 MB/s eta 0:00:01
   ---------------------------------------  2.6/2.6 MB 12.0 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 11.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/28.1 MB ? eta -:--:--
   - -------------------------------------- 1.3/28.1 MB 27.7 MB/s eta 0:00:01
   ---- ----------------------------------- 2.9/28.1 MB 31.0 MB/s eta 0:00:01
   ------ --------------------------------- 4.8/28.1 MB 34.2 MB/s eta 0:00:01
   --------- ------------------------------ 6.7/28.1 MB 35.9 MB/s eta 0:00:01
   ------------ --------------------------- 8.9/28.1 MB 37.9 MB/s eta 0:00:01
   -------

In [29]:
from numba import cuda


cc_cores_per_SM_dict = {
    (2,0) : 32,
    (2,1) : 48,
    (3,0) : 192,
    (3,5) : 192,
    (3,7) : 192,
    (5,0) : 128,
    (5,2) : 128,
    (6,0) : 64,
    (6,1) : 128,
    (7,0) : 64,
    (7,5) : 64,
    (8,0) : 64,
    (8,6) : 128,
    (8,9) : 128,
    (9,0) : 128
    }
# the above dictionary should result in a value of "None" if a cc match 
# is not found.  The dictionary needs to be extended as new devices become
# available, and currently does not account for all Jetson devices
device = cuda.get_current_device()
my_sms = getattr(device, 'MULTIPROCESSOR_COUNT')
my_cc = device.compute_capability
cores_per_sm = cc_cores_per_SM_dict.get(my_cc)
total_cores = cores_per_sm*my_sms
print("GPU compute capability: " , my_cc)
print("GPU total number of SMs: " , my_sms)
print("total cores: " , total_cores)

GPU compute capability:  (8, 6)
GPU total number of SMs:  20
total cores:  2560


In [30]:
from numba import cuda

device = cuda.get_current_device()
attribs = [s for s in dir(device) if s.isupper()]
for attr in attribs:
    print(attr, '=', getattr(device, attr))

COMPUTE_CAPABILITY_MAJOR = 8
COMPUTE_CAPABILITY_MINOR = 6
MULTIPROCESSOR_COUNT = 20


In [31]:
from numba.cuda.cudadrv import enums
from numba import cuda

device = cuda.get_current_device()
attribs= [name.replace("CU_DEVICE_ATTRIBUTE_", "") for name in dir(enums) if name.startswith("CU_DEVICE_ATTRIBUTE_")]
for attr in attribs:
    print(attr, '=', getattr(device, attr))

ASYNC_ENGINE_COUNT = 1
CAN_MAP_HOST_MEMORY = 1
CAN_USE_HOST_POINTER_FOR_REGISTERED_MEM = 0
CLOCK_RATE = 1035000
COMPUTE_CAPABILITY_MAJOR = 8
COMPUTE_CAPABILITY_MINOR = 6
COMPUTE_MODE = 0
COMPUTE_PREEMPTION_SUPPORTED = 1
CONCURRENT_KERNELS = 1
CONCURRENT_MANAGED_ACCESS = 0
COOPERATIVE_LAUNCH = 1
COOPERATIVE_MULTI_DEVICE_LAUNCH = 0
ECC_ENABLED = 0
GLOBAL_L1_CACHE_SUPPORTED = 1
GLOBAL_MEMORY_BUS_WIDTH = 128
GPU_OVERLAP = 1
HOST_NATIVE_ATOMIC_SUPPORTED = 0
INTEGRATED = 0
IS_MULTI_GPU_BOARD = 0
KERNEL_EXEC_TIMEOUT = 1
L2_CACHE_SIZE = 2097152
LOCAL_L1_CACHE_SUPPORTED = 1
MANAGED_MEMORY = 1
MAX_BLOCK_DIM_X = 1024
MAX_BLOCK_DIM_Y = 1024
MAX_BLOCK_DIM_Z = 64
MAX_GRID_DIM_X = 2147483647
MAX_GRID_DIM_Y = 65535
MAX_GRID_DIM_Z = 65535
MAX_MAX_TEXTURE_2D_MIPMAPPED_HEIGHT = 32768
MAX_PITCH = 2147483647
MAX_REGISTERS_PER_BLOCK = 65536
MAX_REGISTERS_PER_MULTIPROCESSOR = 65536
MAX_SHARED_MEMORY_PER_BLOCK = 49152
MAX_SHARED_MEMORY_PER_BLOCK_OPTIN = 101376
MAX_SHARED_MEMORY_PER_MULTIPROCESSOR = 102400
MAX